In [3]:
import boto3
import json
import os
import pprint

In [2]:
BUCKET = 'mediboard-textract'

In [1]:
STUDIES_PATH = os.path.expanduser('pdf')

In [4]:
pdfs = os.listdir(STUDIES_PATH)

In [5]:
pdfs, len(pdfs), STUDIES_PATH

(['COC.0b013e318248d83f.pdf',
  'j.1755-5949.2010.00196.x.pdf',
  'jandrol.111.014514.pdf',
  'dom.12847.pdf',
  'INF.0000000000001647.pdf',
  'beem.2000.0076.pdf',
  'dta.2106.pdf',
  '07357907.2016.1174257.pdf',
  's12895-018-0071-3.pdf',
  'ane.0000000000000931.pdf',
  'jcp.0b013e31822bb390.pdf',
  'ars.2013.5593.pdf',
  'sm2.22.pdf',
  '0022034512454296.pdf',
  'j.preghy.2019.09.012.pdf',
  'j.ijporl.2014.04.014.pdf',
  'mpg.0000000000000872.pdf',
  'jac%252Fdkaa341.pdf',
  'j.mehy.2004.11.026.pdf',
  's12887-016-0547-4.pdf',
  '1352458511419700.pdf',
  'j.ijwd.2015.12.006.pdf',
  'ALN.0b013e31825a2a4e.pdf',
  's11764-019-00786-6.pdf',
  's00198-005-1890-8.pdf',
  'jcp.0b013e3181b591e0.pdf',
  '1471-2482-10-13.pdf',
  'bcr-2016-214396.pdf',
  's11060-015-2020-x.pdf',
  's10637-006-9023-6.pdf',
  '00952990.2014.884102.pdf',
  'MJT.0000000000000695.pdf',
  'bmt.2011.104.pdf',
  'spv.0000000000000343.pdf',
  'j.contraception.2012.05.002.pdf',
  'dss.0000000000002168.pdf',
  'CIRCINTER

In [6]:
import base64

with open("{}/{}".format(STUDIES_PATH, pdfs[0]), "rb") as study_file:
    encoded_string = base64.b64encode(study_file.read())
    print(type(encoded_string))

<class 'bytes'>


In [7]:
textract = boto3.client('textract', region_name='us-west-2')

In [11]:
import boto3
from pprint import pprint
import pathlib
import os

In [12]:
s3 = boto3.client("s3")

In [18]:
pdfs_to_ids = []
progress = 0
for pdf in pdfs:

    s3 = boto3.client("s3")
    # file_name = os.path.join(pathlib.Path(__file__).parent.resolve(), "pdf")

    response = s3.upload_file("pdf/" + pdf, BUCKET, "unstructured/" + pdf)
    # pprint(response)  # prints None

    if response == None:
        response = textract.start_document_text_detection(
            DocumentLocation={
                'S3Object': {
                    'Bucket': 'mediboard-textract',
                    'Name': 'unstructured/' + pdf,
                    # 'Version': 'string'
                }
            },
            # NotificationChannel={
            #     'SNSTopicArn': 'string',
            #     'RoleArn': 'string'
            # },
            OutputConfig={
                'S3Bucket': 'mediboard-textract',
                'S3Prefix': 'structured'
            },
        )

        job_id = response['JobId']

        pdfs_to_ids.append((pdf, job_id))
        progress += 1
        print("Num completed: " + str(progress))

    


Num completed: 1
Num completed: 2
Num completed: 3
Num completed: 4
Num completed: 5
Num completed: 6
Num completed: 7
Num completed: 8
Num completed: 9
Num completed: 10
Num completed: 11
Num completed: 12
Num completed: 13
Num completed: 14
Num completed: 15
Num completed: 16
Num completed: 17
Num completed: 18
Num completed: 19
Num completed: 20
Num completed: 21
Num completed: 22
Num completed: 23
Num completed: 24
Num completed: 25
Num completed: 26
Num completed: 27
Num completed: 28
Num completed: 29
Num completed: 30
Num completed: 31
Num completed: 32
Num completed: 33
Num completed: 34
Num completed: 35
Num completed: 36
Num completed: 37
Num completed: 38
Num completed: 39
Num completed: 40
Num completed: 41
Num completed: 42
Num completed: 43
Num completed: 44
Num completed: 45
Num completed: 46
Num completed: 47
Num completed: 48
Num completed: 49
Num completed: 50
Num completed: 51
Num completed: 52
Num completed: 53
Num completed: 54
Num completed: 55
Num completed: 56
N

In [21]:

open("pdfs_to_ids.json", "w").write(json.dumps(pdfs_to_ids))

67015

In [40]:
response = textract.start_document_text_detection(
    DocumentLocation={
        'S3Object': {
            'Bucket': 'mediboard-textract',
            'Name': 'unstructured/1475-2832-3-2.pdf',
            # 'Version': 'string'
        }
    },
    # NotificationChannel={
    #     'SNSTopicArn': 'string',
    #     'RoleArn': 'string'
    # },
    OutputConfig={
        'S3Bucket': 'mediboard-textract',
        'S3Prefix': 'structured'
    },
)

In [41]:
print(response)
job_id = response['JobId']

{'JobId': 'bc4c820238a6914820fc32846f88bb7ea3db501351ba972b95915152d97038a2', 'ResponseMetadata': {'RequestId': 'e7cda31b-26e5-4e56-a97c-a2e8a28f66ab', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e7cda31b-26e5-4e56-a97c-a2e8a28f66ab', 'content-type': 'application/x-amz-json-1.1', 'content-length': '76', 'date': 'Thu, 10 Nov 2022 00:25:42 GMT'}, 'RetryAttempts': 0}}


In [ ]:
# TODO(davonprewitt): Create 

In [165]:
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    # print(type(string))
    if type(string) != str:
        return False
    try: 
        # print("H", string)
        # print("i", parse(string, fuzzy=fuzzy))
        return True

    except ValueError:
        return False

In [198]:
import datetime as dt

def is_date_2(string):

    fmts = ('%d %B %Y', '%B %d %Y')
    if type(string) != str:
        return False

    print("HEY")
    for fmt in fmts:
        try:
            print("BEFORE")
            t = dt.datetime.strptime(string, fmt)
            print("AFTER")
            return True
        except ValueError as err:
            pass

    return False


In [221]:
import datefinder

def is_date_3(string):
    if type(string) != str:
        return []

    matches = datefinder.find_dates(string, strict=True)
    return [match for match in matches]

In [42]:
s3 = boto3.client('s3', region_name='us-west-2')
# print(s3.list_objects(Bucket=BUCKET)['Contents'])

# dict_keys(
# ['BlockType', 
# 'ColumnIndex', 
# 'ColumnSpan',
# 'Confidence', 
# 'EntityTypes', 
# 'Geometry', 
# 'Hint', 
# 'Id', 
# 'Page', 
# 'PageClassification', 
# 'Query', 
# 'Relationships', 
# 'RowIndex', 
# 'RowSpan', 
# 'SelectionStatus', 
# 'Text', 
# 'TextType'
# ])
def flatten(l):
    return [item for sublist in l for item in sublist]

annotations = []
for key in s3.list_objects(Bucket=BUCKET)['Contents']:
    object = key['Key']
    if 'structured/{}'.format(job_id) in object and '.s3_access_check' not in object:
#         print(object)
        data = s3.get_object(Bucket=BUCKET, Key=object)
        contents = data['Body'].read()
        blocks = json.loads(contents.decode("utf-8"))['Blocks']
        blocks = [block for block in blocks if block['Page'] == 1]
        # print(parsed)
        annotations.extend(blocks)
        # parsed
        # [annotations.extend(block) for block in blocks]
        # print(parsed)
print(len(annotations))
        
#         title = ' '.join([p['Text'] for p in parsed['Blocks'] if p['BlockType'] == "LINE" and p['Geometry']['BoundingBox']['Height'] > .018]).lstrip('Clinical Study')
#         date_pos = [is_date_3(p['Text']) for p in parsed['Blocks'] if len(is_date_3(p['Text'])) > 0]
#         # print(flatten(date_pos)[-1])
#         # print(title)
#         for p in parsed['Blocks']:
#             if (len(p['Relationships'][0]['Ids']) > 14 and 'Copyright'):
#                 print(p['Text'])
#             # else:
#             #     pass
        
#         break


528


In [10]:
from Bio import Entrez
Entrez.email = "davonprewitt@gmail.com"     # Always tell NCBI who you are
handle = Entrez.esearch(db="pubmed", term="A Double-blind, Randomized, Placebo-Controlled Trial of Fluoxetine in Children and Adolescents With Depression", retmax="5" )
record = Entrez.read(handle)

# for doc in records:
#     print(doc)
record

{'Count': '19', 'RetMax': '5', 'RetStart': '0', 'IdList': ['35033635', '32460523', '25721187', '24552282', '24350814'], 'TranslationSet': [{'From': 'Double-blind,', 'To': '"double-blind method"[MeSH Terms] OR ("double-blind"[All Fields] AND "method"[All Fields]) OR "double-blind method"[All Fields] OR ("double"[All Fields] AND "blind"[All Fields]) OR "double blind"[All Fields]'}, {'From': 'Randomized,', 'To': '"random allocation"[MeSH Terms] OR ("random"[All Fields] AND "allocation"[All Fields]) OR "random allocation"[All Fields] OR "random"[All Fields] OR "randomization"[All Fields] OR "randomized"[All Fields] OR "randomisation"[All Fields] OR "randomisations"[All Fields] OR "randomise"[All Fields] OR "randomised"[All Fields] OR "randomising"[All Fields] OR "randomizations"[All Fields] OR "randomize"[All Fields] OR "randomizes"[All Fields] OR "randomizing"[All Fields] OR "randomness"[All Fields] OR "randoms"[All Fields]'}, {'From': 'Trial', 'To': '"clinical trials as topic"[MeSH Terms

In [11]:
from pymed import PubMed
pubmed = PubMed(tool="PubMedSearcher", email="myemail@ccc.com")

## PUT YOUR SEARCH TERM HERE ##
search_term = "A Double-blind, Randomized, Placebo-Controlled Trial of Fluoxetine in Children and Adolescents With Depression[title]"
results = pubmed.query(search_term, max_results=10)
articleList = []
articleInfo = []

for article in results:
# Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
# We need to convert it to dictionary with available function
    articleDict = article.toDict()
    articleList.append(articleDict)

# Generate list of dict records which will hold all article details that could be fetch from PUBMED API
for article in articleList:
#Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
    pubmedId = article['pubmed_id'].partition('\n')[0]
    print(article.keys())
    # Append article info to dictionary 
    articleInfo.append({u'pubmed_id':pubmedId,
                       u'title':article['title'],
                       u'keywords':article['keywords'],
                       u'journal':article['journal'],
                       u'abstract':article['abstract'],
                       u'conclusions':article['conclusions'],
                       u'methods':article['methods'],
                       u'results': article['results'],
                       u'copyrights':article['copyrights'],
                       u'doi':article['doi'],
                       u'publication_date':article['publication_date'], 
                       u'authors':article['authors']})


dict_keys(['pubmed_id', 'title', 'abstract', 'keywords', 'journal', 'publication_date', 'authors', 'methods', 'conclusions', 'results', 'copyrights', 'doi', 'xml'])


In [ ]:


def get_article_info(title):
    pass

'''
1. Parse PDF using Textract
2. Get title using rules-based approach
3. Check if in database already -> yes, request -> else pipeline
4. Search pubmed about information

'''
# def to_dict(self):
#     return {
#         'id': self.id, X
#         'upload_date': self.upload_date,   X
#         *** 'short_title': self.short_title,   X
#         'official_title': self.official_title, X
#         'description': self.description, X
#         *** 'responsible_party': self.responsible_party, 
#         'sponsor': self.sponsor,
#         'type': str(self.type),  -> classifier maybe
#         'purpose': str(self.purpose),
#         'intervention_type': str(self.intervention_type),  
#         'min_age': self.min_age,
#         'min_age_units': str(self.min_age_units),
#         'max_age': self.max_age,
#         'min_age_units': str(self.max_age_units),
#         'gender': str(self.gender),
#         'criteria': [x.to_dict() for x in self.criteria],
#         'measures': [x.to_dict() for x in self.measures],
#         'analytics': [x.to_dict() for x in self.analytics],

# NLP = textract -> text + spatial -> 
# can we try rules based first?
#         'baselines': [x.to_dict() for x in self.baselines],
#         'groups': [x.to_dict() for x in self.groups],
#         'conditions': [x.conditions.to_dict() for x in self.conditions],
#         'treatments': [x.treatments.to_dict() for x in self.treatments]
#     }


In [12]:
print(len(articleInfo))
[print(article['title']) for article in articleInfo]
articleInfo

1
A double-blind, randomized, placebo-controlled trial of fluoxetine in children and adolescents with depression.


[{'pubmed_id': '9366660',
  'title': 'A double-blind, randomized, placebo-controlled trial of fluoxetine in children and adolescents with depression.',
  'keywords': [],
  'journal': 'Archives of general psychiatry',
  'abstract': 'Depression is a major cause of morbidity and mortality in children and adolescents. To date, randomized, controlled, double-blind trials of antidepressants (largely tricyclic agents) have yet to reveal that any antidepressant is more effective than placebo. This article is of a randomized, double-blind, placebo-controlled trial of fluoxetine in children and adolescents with depression.\nNinety-six child and adolescent outpatients (aged 7-17 years) with nonpsychotic major depressive disorder were randomized (stratified for age and sex) to 20 mg of fluoxetine or placebo and seen weekly for 8 consecutive weeks. Randomization was preceded by 3 evaluation visits that included structured diagnostic interviews during 2 weeks, followed 1 week later by a 1-week, sing